In [56]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from data_loading import read_table_to_df, write_df_to_table
import pickle

In [57]:
df = read_table_to_df(table_name="USER_SIM_DATASET") 
orig = df.copy()
orig

Data from USER_SIM_DATASET loaded successfully!


,RowNumber,CustomerId,Surname,Age,EstimatedSalary,Balance,CreditScore,NumOfProducts,Geography,Gender,HasCrCard,IsActiveMember
0,6528,15722692,Kazakova,38,75574.48,116439.65,464.0,1,France,Male,True,False
1,9760,15790093,Aguirre,27,125451.01,0.00,627.0,2,France,Female,True,False
2,6850,15775678,Uspensky,44,152108.47,0.00,716.0,1,France,Female,True,True
3,1795,15606841,Ibbott,38,156603.70,0.00,823.0,2,France,Male,True,False
4,8534,15768566,K?,34,37479.97,0.00,706.0,2,France,Male,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
4996,4484,15774192,Miller,38,80575.44,105435.74,539.0,1,Germany,Female,False,False
4997,2432,15782247,Yeh,22,186233.26,0.00,540.0,3,France,Male,True,True
4998,4593,15658475,Lori,36,89983.02,142882.49,834.0,1,France,Male,True,False
4999,2782,15641416,Shaffer,61,157527.60,94867.18,732.0,2,Germany,Female,True,True


In [58]:
df = df.drop(columns=['RowNumber','CustomerId','Surname'])
numerical_cols = ['Age', 'EstimatedSalary', 'Balance','CreditScore', 'NumOfProducts'] 
categorical_cols = ['Geography','Gender', 'HasCrCard', 'IsActiveMember']

In [59]:
df['HasCrCard'] = df['HasCrCard'].astype(int)
df['IsActiveMember'] = df['IsActiveMember'].astype(int)

In [60]:
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Geography'] = label_encoder.fit_transform(df['Geography'])

In [61]:
min_value = df['CreditScore'].min() - 0.001
max_value = df['CreditScore'].max() + 0.001

num_categories = 3 

bins = np.linspace(min_value, max_value, num_categories + 1)
labels = ['1', '2', '3'] 

df['Segment'] = pd.cut(df['CreditScore'], bins=bins, labels=labels, right=False)

df

,Age,EstimatedSalary,Balance,CreditScore,NumOfProducts,Geography,Gender,HasCrCard,IsActiveMember,Segment
0,38,75574.48,116439.65,464.0,1,0,1,1,0,1
1,27,125451.01,0.00,627.0,2,0,0,1,0,2
2,44,152108.47,0.00,716.0,1,0,0,1,1,3
3,38,156603.70,0.00,823.0,2,0,1,1,0,3
4,34,37479.97,0.00,706.0,2,0,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...
4996,38,80575.44,105435.74,539.0,1,1,0,0,0,2
4997,22,186233.26,0.00,540.0,3,0,1,1,1,2
4998,36,89983.02,142882.49,834.0,1,0,1,1,0,3
4999,61,157527.60,94867.18,732.0,2,1,0,1,1,3


In [62]:
with open('model_pickles/XGB_Classifier.pkl', 'rb') as file:
    model = pickle.load(file)

probabilities = model.predict_proba(df)

prob_df = pd.DataFrame(probabilities, columns=['Probability of 0', 'Probability of 1'])

orig['Probability of 0']=(prob_df['Probability of 0'] * 10).round() / 10 
orig['Probability of 1']=(prob_df['Probability of 1'] * 10).round() / 10 
orig


,RowNumber,CustomerId,Surname,Age,EstimatedSalary,Balance,CreditScore,NumOfProducts,Geography,Gender,HasCrCard,IsActiveMember,Probability of 0,Probability of 1
0,6528,15722692,Kazakova,38,75574.48,116439.65,464.0,1,France,Male,True,False,0.0,1.0
1,9760,15790093,Aguirre,27,125451.01,0.00,627.0,2,France,Female,True,False,0.1,0.9
2,6850,15775678,Uspensky,44,152108.47,0.00,716.0,1,France,Female,True,True,0.7,0.3
3,1795,15606841,Ibbott,38,156603.70,0.00,823.0,2,France,Male,True,False,0.9,0.1
4,8534,15768566,K?,34,37479.97,0.00,706.0,2,France,Male,True,True,0.9,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,4484,15774192,Miller,38,80575.44,105435.74,539.0,1,Germany,Female,False,False,0.0,1.0
4997,2432,15782247,Yeh,22,186233.26,0.00,540.0,3,France,Male,True,True,0.0,1.0
4998,4593,15658475,Lori,36,89983.02,142882.49,834.0,1,France,Male,True,False,0.4,0.6
4999,2782,15641416,Shaffer,61,157527.60,94867.18,732.0,2,Germany,Female,True,True,0.4,0.6


In [63]:
counts = orig['Probability of 1'].value_counts().sort_index()
counts

Probability of 1
0.1     431
0.2     105
0.3      76
0.4      24
0.5      38
0.6     738
0.7     250
0.8     145
0.9     968
1.0    2226
Name: count, dtype: int64

In [64]:
write_df_to_table(orig, "CHURN_PROBA")

DataFrame written to CHURN_PROBA successfully!


In [65]:
orig.to_excel('CHURN_PROBA.xlsx', index=False)